This notebook cleans and merges the regional UK HPI dataset with the 8 macroeconomic indicators and Google Trends indices. The link to each of the datasets can be found here:
1. HPI: https://drive.google.com/file/d/1zTPNSV2soU48ksW86vaEVD4nzVzXnHz1/view?usp=sharing
2. Average Earnings: https://drive.google.com/file/d/1Yc-3Zyomyav2_dg6CMEA52f4w6XDAhSn/view?usp=sharing
3. CPI: https://drive.google.com/file/d/1j4I80vJvqcB_BusUaUhk5G0M08i-JZJZ/view?usp=sharing
4. Unemployment Rate: https://drive.google.com/file/d/1SowP8X87_dsjVl_379eO8D49tGAxQs_l/view?usp=sharing
5. Bank Rate: https://drive.google.com/file/d/1CQy4uyLEFJhXpOp3PsxNqukGW2YIgHKR/view?usp=sharing
6. Mortgage Rate: https://drive.google.com/file/d/16zJaQ-vCZ5PJ3JHoIG0bQbeGmOgYa_sZ/view?usp=sharing
7. Mortgage Approval: https://drive.google.com/file/d/1LMjfa5Rx3p-ERMm65nvsQlNhSNTwzb-C/view?usp=sharing
8. Consumer Confidence Index: https://drive.google.com/file/d/1_7l8sQVxRFcxUTR1_PG1FzCoL9HsXeG0/view?usp=sharing
9. Construction Cost Index: https://drive.google.com/file/d/1wEoM0Ub7QzAHBVg94t3Y6btxchZKtA85/view?usp=sharing
10. Google Trend indices: https://drive.google.com/file/d/1sDR65v62YxxgPUVbcRtRcCRRVBn2TLmu/view?usp=sharing

In [ ]:
# Merging HPI (regional) + macro series + Google Trends
!pip -q install XlsxWriter

from google.colab import drive, files
drive.mount("/content/drive")

import pandas as pd
from pathlib import Path
from functools import reduce


# Defining base directories

BASE_DIR = Path("/content/drive/MyDrive/Datasets")
OUT_DIR  = BASE_DIR / "MERGED_OUTPUTS"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Finding the files by partial filename

def find_file(*fragments):
    fragments = [f.lower() for f in fragments]
    for path in BASE_DIR.iterdir():
        if path.is_file() and all(f in path.stem.lower() for f in fragments):
            return path
    raise FileNotFoundError(f"Required file not found for fragments: {fragments}")


# Finding all required datasets

paths = {
    "HPI"      : find_file("hpi_clean_panel"),
    "AWE"      : find_file("averageearnings_monthly_cleaned_2005_2025"),
    "UNEMP"    : find_file("unemploymentrate_monthly_cleaned_2005_2025"),
    "CPI"      : find_file("cpi_monthly_cleaned_2005_2025"),
    "APPR"     : find_file("mortgageapprovals_monthly_cleaned_2005_2025"),
    "MORT2Y"   : find_file("mortgagerate_monthly_cleaned_2005_2025"),
    "BANKRATE" : find_file("bankrate_monthly_cleaned_2005_2025"),
    "CONF"     : find_file("ConsumerConfidence_monthly_cleaned_2005_2025_modified"),
    "CONST"    : find_file("BuildingMaterials_monthly_cleaned_2005_2025"),
    "TRENDS"   : find_file("theme_trends_monthly_standardised_modified"),
}


def read_any(path: Path) -> pd.DataFrame:
    """Read CSV or Excel with encoding fallback."""
    if path.suffix.lower() in (".xlsx", ".xls"):
        return pd.read_excel(path)
    try:
        return pd.read_csv(path)
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="latin1")

def to_month_start_mdy(series: pd.Series) -> pd.Series:
    """Parse MDY dates and align to month start."""
    return (
        pd.to_datetime(series, errors="coerce", dayfirst=False)
          .dt.to_period("M")
          .dt.to_timestamp(how="start")
    )

def clean_numeric(df: pd.DataFrame) -> pd.DataFrame:
    """Strip %, commas, and coerce all non-Date columns to numeric."""
    for col in df.columns:
        if col == "Date":
            continue
        df[col] = (
            df[col].astype(str)
                  .str.replace("%", "", regex=False)
                  .str.replace(",", "", regex=False)
                  .str.extract(r"([-+]?\d*\.?\d+)")[0]
                  .astype(float)
        )
    return df

def load_macro_series(path: Path, rename=None, prefix=None) -> pd.DataFrame:
    """
    Load a monthly macro series:
    - Parse MDY dates
    - Clean numerics
    - Deduplicate to one observation per month
    """
    df = read_any(path).copy()

    date_col = "Date" if "Date" in df.columns else df.columns[0]
    df["Date"] = to_month_start_mdy(df[date_col])
    df = df.dropna(subset=["Date"])

    # Removing redundant year/month columns
    df = df.drop(columns=[c for c in df.columns if c.lower() in ("year", "month") and c != "Date"],
                 errors="ignore")

    df = df[["Date"] + [c for c in df.columns if c != "Date"]]
    df = clean_numeric(df)

    if rename:
        df = df.rename(columns=rename)
    if prefix:
        df = df.rename(columns={c: f"{prefix}{c}" for c in df.columns if c != "Date"})

    return (
        df.sort_values("Date")
          .groupby("Date", as_index=False)
          .agg({c: "last" for c in df.columns if c != "Date"})
    )


# Loading and standardising macroeconomic indicators

awe   = load_macro_series(paths["AWE"], {
    "TotalWeeklyEarnings": "AWE_Total",
    "RegularWeeklyEarnings": "AWE_Regular",
    "awe_total": "AWE_Total",
    "awe_regular": "AWE_Regular"
})

unemp = load_macro_series(paths["UNEMP"], {"UnemploymentRate": "UnemploymentRate"})
cpi   = load_macro_series(paths["CPI"],   {"CPI": "CPI"})
appr  = load_macro_series(paths["APPR"],  {"MortgageApprovals": "MortgageApprovals"})
mort2 = load_macro_series(paths["MORT2Y"],{"MortgageRate": "MortgageRate_2YFix"})
br    = load_macro_series(paths["BANKRATE"], {"BankRate": "BankRate"})
conf  = load_macro_series(paths["CONF"], {"ConsumerConfidence": "ConsumerConfidence"})
const = load_macro_series(paths["CONST"], prefix="BM_")

# Google Trends theme indices
trends = load_macro_series(paths["TRENDS"])
trends = trends.rename(
    columns={c: (c if c.lower().startswith("gt_") else f"gt_{c}")
             for c in trends.columns if c != "Date"}
)

# Merging all macro series by month
macro = reduce(lambda l, r: pd.merge(l, r, on="Date", how="outer"),
               [awe, unemp, cpi, appr, mort2, br, conf, const, trends])


# Loading and preparing regional HPI data

hpi_raw = read_any(paths["HPI"]).copy()

if "Date" not in hpi_raw.columns:
    raise ValueError("HPI dataset must contain a 'Date' column.")

hpi_raw["Date"] = to_month_start_mdy(hpi_raw["Date"])

# Normalising region identifiers
for col in hpi_raw.columns:
    cl = col.lower()
    if "region" in cl and "name" in cl and "RegionName" not in hpi_raw.columns:
        hpi_raw = hpi_raw.rename(columns={col: "RegionName"})
    if cl in ("areacode", "area_code", "code") and "AreaCode" not in hpi_raw.columns:
        hpi_raw = hpi_raw.rename(columns={col: "AreaCode"})

keep_cols = [c for c in ["Date", "RegionName", "AreaCode",
                         "AveragePrice", "Index", "SalesVolume"]
             if c in hpi_raw.columns]

hpi = hpi_raw[keep_cols].sort_values(["RegionName", "Date"])


# Filtering for the study window and merging regional HPI with macro data

START, END = pd.Timestamp(2005, 1, 1), pd.Timestamp(2025, 6, 1)

macro = macro[(macro["Date"] >= START) & (macro["Date"] <= END)]
hpi   = hpi[(hpi["Date"]   >= START) & (hpi["Date"]   <= END)]

hpi_regional = pd.merge(hpi, macro, on="Date", how="left", validate="m:1")


# Save outputs

xlsx_path = OUT_DIR / "HPI_regional_merged_2005_2025_ddmmyyyy_MODIFIED.xlsx"
csv_path  = OUT_DIR / "HPI_regional_merged_2005_2025_ddmmyyyy_MODIFIED.csv"

with pd.ExcelWriter(xlsx_path, engine="xlsxwriter", datetime_format="dd/mm/yyyy") as writer:
    hpi_regional.to_excel(writer, index=False, sheet_name="HPI_Regional")
    worksheet = writer.sheets["HPI_Regional"]
    workbook  = writer.book
    worksheet.set_column(
        hpi_regional.columns.get_loc("Date"),
        hpi_regional.columns.get_loc("Date"),
        12,
        workbook.add_format({"num_format": "dd/mm/yyyy"})
    )

csv_tmp = hpi_regional.copy()
csv_tmp["Date"] = csv_tmp["Date"].dt.strftime("%d/%m/%Y")
csv_tmp.to_csv(csv_path, index=False)

print("Saved Excel:", xlsx_path)
print("Saved CSV  :", csv_path)
files.download(str(xlsx_path))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved Excel: /content/drive/MyDrive/Datasets/MERGED_OUTPUTS/HPI_regional_merged_2005_2025_ddmmyyyy_MODIFIED.xlsx
Saved CSV  : /content/drive/MyDrive/Datasets/MERGED_OUTPUTS/HPI_regional_merged_2005_2025_ddmmyyyy_MODIFIED.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>